# Assignment 1

**Group 22**

A. Siganos ( 1283871 )     
J. Gómez Robles ( 1286552 )

In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

# To measure running times
import time
#from matplotlib import pyplot
import matplotlib.pyplot as plt

# Only for the word2vec parsing of last exercise (loading word2vec with gensim was part of the initial template)
import gensim


Using TensorFlow backend.


In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix


## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [3]:
################################################################################
############################## 0. Util functions ###############################
################################################################################

# Run this block first
def getKeyFromValue(dictionary, target):
    return list(dictionary.keys())[list(dictionary.values()).index(target)]

plot_labels = []

def update_annot(ind):
    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}, {}".format(" ".join(list(map(str,ind["ind"]))), 
                           " ".join([plot_labels[n] for n in ind["ind"]]))
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
    annot.get_bbox_patch().set_alpha(0.4)


def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()


# Visualization function
def plot_embedding(x, labels, title):
    fig,ax = plt.subplots()
    fig.set_size_inches(18.5, 10.5)
    plt.title(title, fontsize=20)
    sc = plt.scatter(x[:,0],x[:,1], s=5)

    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                        bbox=dict(boxstyle="round", fc="w"),
                        arrowprops=dict(arrowstyle="->"))
    annot.set_visible(False)

    fig.canvas.mpl_connect("motion_notify_event", hover)
    plt.show()




In [4]:
################################################################################
################# 1. Implement word-word co-ocurrence matrix ###################
################################################################################

# Create my co-ocurrence matrix, initially 0 (VxV size)
# How to access this matrix:
#   Each column is an (index - 1) (column 0 is word in index 1; column 1 is word in index 2; ...)
wcoMatrix = np.zeros([V, V]) 

# Using my window_size_corpus to define my context scope
scope = window_size_corpus

# TODO: Do not make that many iterations
# Greedy approach first (to be able to compare the optimization)

# For each line in the corpus. Note that they preserve the order, even when they are indexes now.
for s in corpus:
    # For each word
    for current_index in range(0, len(s)):
        current_value = s[current_index] # Get the 'word'
        # From left to right
        for neighbor_index in range(current_index - scope, current_index + scope + 1):
            # Never out of boundaries and never the same index
            if ( neighbor_index >= 0 and neighbor_index < len(s) ) and ( neighbor_index != current_index ):
                # Get my neighbor 'word'
                neighbor = s[neighbor_index] 
                # Never myself and myself (keeping diagonal to 0)
                if current_value != neighbor:
                    # Update the ocurrence
                    #wcoMatrix[current_value - 1, neighbor - 1] += 1
                    wcoMatrix[current_value, neighbor] += 1
                    
################################################################################
################################# 2. Normalize #################################
################################################################################
wcoMatrix = wcoMatrix / wcoMatrix.max()
wcoMatrix

print("Sumary of the word-word co-ocurrence matrix")
print(" - Shape", wcoMatrix.shape)
print(" - Min value", wcoMatrix.min())
print(" - Max value", wcoMatrix.max())


Sumary of the word-word co-ocurrence matrix
 - Shape (1183, 1183)
 - Min value 0.0
 - Max value 1.0


In [5]:
################################################################################
########### 3. Compute cosine similarity to Alice, Dinah and Rabbit ############
################################################################################

words_to_compare = ["Alice", "Dinah", "Rabbit"]

# Get my index
full_index = tokenizer.word_index

# Iterate on all possible combinations
for w1 in words_to_compare:
    w1 = w1.lower() 
    x = full_index[w1]
    for w2 in words_to_compare:
        w2 = w2.lower()
        if w1 != w2:
            y = full_index[w2]
            X = wcoMatrix[x, :].reshape((1, V))
            Y = wcoMatrix[y, :].reshape((1, V))
            print("cosine_similarity(%s, %s)=%s" % (w1, w2, cosine_similarity(X, Y)))
            

cosine_similarity(alice, dinah)=[[0.39360011]]
cosine_similarity(alice, rabbit)=[[0.47890931]]
cosine_similarity(dinah, alice)=[[0.39360011]]
cosine_similarity(dinah, rabbit)=[[0.29862324]]
cosine_similarity(rabbit, alice)=[[0.47890931]]
cosine_similarity(rabbit, dinah)=[[0.29862324]]


In [6]:
################################################################################
################## 4a. List the 5 closest words to 'Alice'. ####################
################################################################################

################################
### We use Nearest Neighbors ###
################################

# It was not importing
from sklearn.neighbors import NearestNeighbors

# Define our variables
targetWord = "Alice".lower()
X = wcoMatrix
N_NEIGHBORS = 5
full_index = tokenizer.word_index

# Perfrom the NearestNeibhbors algorithm
# TODO: Tune parameters
nbrs = NearestNeighbors(n_neighbors = N_NEIGHBORS).fit(X)

# Get the index corresponding to the target word
aliceMatrixIndex = full_index[targetWord]

# Get results from nearest neighbors
distances, indices = nbrs.kneighbors()
aliceIndices = indices[aliceMatrixIndex, :]
aliceDistances = distances[aliceMatrixIndex, :]

# Output result
print("Nearest neighbors (co-ocurrence matrix based) to %s (index %s):" % (targetWord, aliceMatrixIndex))
for i in range(0, N_NEIGHBORS):
    print( "%s [index %s] with distance: %s" % (getKeyFromValue(full_index, aliceIndices[i]), aliceIndices[i], aliceDistances[i]) )

# TODO: Think about what we are computing. 
# With first approach, a neighbor would be a word with a similar vector in the co-ocurrence matrix, 
# not a word within the same context (window_size). Although words in the similar window_size should have a
# similar vector, isn't?

Nearest neighbors (co-ocurrence matrix based) to alice (index 11):
her [index 14] with distance: 0.5616433150108668
that [index 13] with distance: 0.5692975698101006
herself [index 41] with distance: 0.5700573426956416
for [index 18] with distance: 0.5811858638252018
on [index 21] with distance: 0.5837124477057559


**4b. Discuss the results**

We observe that the most similar words are possesives and prepositions. Although these results intuitively make some sense (Alice is the main character of the book and consequently, describing her behaviour is expected), no idiomatic relations can be achieved by means of the simple word-word co-ocurrence matrix.

Furthermore, the minimal distance (cosine similarity normalized) found for alice is of 0.5616, which suggests a poor result - one would be interested on a similarity close to 0.


**5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions? **

TODO. Add some ideas!

Jorge's ideas: sparse matrix VxV size (computation time increases).



In [7]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = wcoMatrix
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()


In [8]:
#reopen your file as follows
# TODO: Check! It is not reloading the file D:
# co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)


## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


# CBOW

In [8]:
#prepare data for cbow
def generate_data_cbow(corpus, window_size, V):
    all_in = [] 
    all_out = []
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)
            if contexts != []:
                all_in.append(sequence.pad_sequences(contexts, maxlen = maxlen))
                all_out.append(np_utils.to_categorical(labels, V))
    return (all_in,all_out) # return the values

            

In [9]:
#create CBOW model

def getCbowModel(dim = 50, extra_layer = False, activation_extra_layer = 'softmax'):

    # The Sequential model is a linear stack of layers.
    cbow = Sequential()
    cbow.add(Embedding(input_dim = V, output_dim = dim, input_length = window_size*2))

    # TODO!!! 
    if extra_layer :
        cbow.add(Dense(input_dim = V, units=dim, kernel_initializer='uniform', activation = activation_extra_layer))
    
    cbow.add( Lambda(lambda x: K.mean(x, axis = 1), output_shape = (dim,)) )
    cbow.add( Dense(V, activation = 'softmax') )

    # define loss function for Cbow
    cbow.compile(loss='categorical_crossentropy', optimizer = 'adadelta')

    return cbow


In [186]:
#train model

################################################################################
############################ 1. Using CBOW model ###############################
################################################################################

dims = [50, 150, 300]
iterations = 2

for dimension in dims:
    print("Running CBOW (dim = %s)..."% dimension)
    cbow = getCbowModel(dimension, False)
    X, Y = generate_data_cbow(corpus, window_size, V)
    for ite in range(iterations):
        loss = 0.
        for x, y in zip(X, Y):
            loss += cbow.train_on_batch(x, y)
        print(ite, loss)

    end_time = time.time()
    
    print("--- %s seconds ---" % (end_time - start_time))

cbowWeights = cbow.get_weights()[0]



Running CBOW (dim = 50)...
0 41684.12801241875
1 39103.43466985226
--- 626.8190407752991 seconds ---
Running CBOW (dim = 150)...
0 41622.70653629303
1 38772.33691751957
--- 733.5874888896942 seconds ---
Running CBOW (dim = 300)...
0 41559.061150312424
1 38503.73327308893
--- 918.9292860031128 seconds ---


# Skipgram

In [9]:
#prepare data for Skipgram

# This function creates a list where _each word_ in the text file (with non-empty content)
# is encoded into two arrays:
#   - x: an array of the size of all his windows_size neighbors with the value of the word (i.e, repeated values)
#   - y: an array of the same size as x pointing to each neighbor. Finally encoded in a |V|-list as one-hot enc
def generate_data_skipgram(corpus, window_size, V):
    all_in = [] 
    all_out = []
    for sentence in corpus: # For, sentence in corpus
        L = len(sentence) # Limit for the iteration (per sentence)

        # For each pair in the index
        for index, word in enumerate(sentence):
            # Create X and Y
            in_words = [] # list of lists
            labels = [] # list of integers

            # Iterate over all the sentence, for each word in the line. I.e, for each word:
            for i in range(index - window_size, index + window_size + 1):
                # If between the limits and not myself
                if i != index and (0 <= i < L):
                    # This means: Associating the words in my neighbor (context) 
                    in_words.append([word]) # Current word (as a list) is associated to...
                    labels.append(sentence[i]) # ... this word (as a simple integer).
                    
            # Once I have saved the context for the current word...
            # ... if there is something in the arrays
            if in_words != []:
                # Save the information to the all_in/out arrays
                all_in.append(np.array(in_words,dtype=np.int32)) # X in integer format
                # Set Y to categorical values. If my output is ([6, 26]). Then I create
                # a vector from 0 to V and mark 6 and 26 as 1
                all_out.append(np_utils.to_categorical(labels, V))
    return (all_in,all_out) # return the values



In [10]:
# create Skipgram model

def getSkipgramModel(dim = 50, extra_layer = False, activation_extra_layer = 'softmax'):

    # The Sequential model is a linear stack of layers.
    skipgram = Sequential()
    
    # First layer (Projection layer. Embedding for wt)
    # 
    # Parameters: 
    #   - input_dim: V (From the book: 1xV (input layer))
    #   - embeddings_initializer = Glorot uniform. Is it for the uniform distribution of the noises?
    #   - input_length: 1. It is needed for the dense layer (https://keras.io/layers/embeddings/#embedding)
    # Input: One-hot ecoding of word wt in the 1xV space
    # Output: vj in the dimension space 1xd
    skipgram.add( Embedding(input_dim = V, output_dim = dim, embeddings_initializer = 'glorot_uniform', input_length = 1) )

    # Intermediate layer. Can be seen as a helper only.
    skipgram.add( Reshape((dim, )) )

    
    # Extra layer
    #
    # Justification: 
    #
    #
    if extra_layer :
        skipgram.add(Dense(input_dim = dim, units = V, kernel_initializer = 'uniform', activation = activation_extra_layer))

    # Last Layer (Output layer. Probabilities of context words)
    # From documentation, Dense implements the operation: 
    #      output = activation(dot(input, kernel) + bias) 
    #   where:
    #      activation is the element-wise activation function passed as the activation argument
    #      kernel is a weights matrix created by the layer
    #      and bias is a bias vector created by the layer (only applicable if use_bias is True). _WE DO NOT SET THIS_.
    #
    # Hence, we are computing ck*vj
    #
    # Parameters:
    #   - input_dim: dim. From embedding layer, dx1
    #   - units: V. The output dimension, From the book 1xV
    #   - kernel_initializer: uniform (?)
    #   - activation: softmax (as described in the book)

    skipgram.add( Dense(input_dim = dim, units = V, kernel_initializer = 'uniform', activation = 'softmax') )
    
    
    # define loss function for Skipgram
    skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta')

    return skipgram

In [11]:
################################################################################
########################## 2. Using Skipgram model #############################
################################################################################

dims = [50, 150, 300]
iterations = 8

for dimension in dims:
    print("Running Skipgram (dim = %s)..."% dimension)
    skipgram = getSkipgramModel(dimension, False)
    #print(skipgram.summary())
    X, Y = generate_data_skipgram(corpus, window_size, V)

    start_time = time.time()
    for ite in range(iterations):
        loss = 0.
        for x, y in zip(X,Y):
            loss += skipgram.train_on_batch(x, y)
        print(ite, loss)
    end_time = time.time()
    
    print("--- %s seconds ---" % (end_time - start_time))
    
skipgramWeights = skipgram.get_weights()[0]


Running Skipgram (dim = 50)...
0 41271.35414361954
1 39126.83639883995
2 39288.24052977562
3 39363.89170718193
4 39432.41858673096
5 39511.995033741
6 39608.390723228455
7 39723.89541876316
--- 153.39438009262085 seconds ---
Running Skipgram (dim = 150)...
0 41222.66726255417
1 38923.405826091766
2 38988.34033322334
3 39005.65833210945
4 39038.85262310505
5 39093.713930249214
6 39156.146334052086
7 39216.637645840645
--- 323.76494121551514 seconds ---
Running Skipgram (dim = 300)...
0 41159.68821120262
1 38730.843128442764
2 38713.53500235081
3 38676.768409490585
4 38648.03497707844
5 38631.883412480354
6 38611.77797353268
7 38578.92355453968
--- 637.9695401191711 seconds ---


# CBOW additional hidden layer

In [60]:
################################################################################
################### 3. Using Skipgram model (extra layer) ######################
################################################################################

#dims = [50, 150, 300]
dims = [50]
iterations = 2

activation_rules = ["softmax", "elu", "selu", "softplus", "softsign", "relu", "tanh", "sigmoid", "hard_sigmoid", "linear"]
for dimension in dims:
    for activation_rule in activation_rules:
        print("Running Skipgram (dim = %s) with activation rule '%s'..."% (dimension, activation_rule))
        try:
            # set extra_layer = true
            cbow = getCbowModel(dimension, True, activation_rule)
            X, Y = generate_data_cbow(corpus, window_size, V)

            start_time = time.time()
            for ite in range(iterations):
                loss = 0.
                for x, y in zip(X,Y):
                    loss += cbow.train_on_batch(x, y)
                print(ite, loss)
            end_time = time.time()

            print("--- %s seconds ---" % (end_time - start_time))
        except:
            print("Error with", activation_rule)

cbowHiddenLayerWeights = cbow.get_weights()[0]



Running Skipgram (dim = 50) with activation rule 'softmax'...
0 41624.25065732002
1 39531.59989595413
--- 49.01548528671265 seconds ---
Running Skipgram (dim = 50) with activation rule 'elu'...
0 40056.498475790024
1 38624.99711847305
--- 34.012505769729614 seconds ---
Running Skipgram (dim = 50) with activation rule 'selu'...
0 39877.45523810387
1 38574.679033041
--- 33.73241710662842 seconds ---
Running Skipgram (dim = 50) with activation rule 'softplus'...
0 39924.52046144009
1 39497.91260719299
--- 38.39311408996582 seconds ---
Running Skipgram (dim = 50) with activation rule 'softsign'...
0 40358.41834950447
1 38613.81048035622
--- 38.05812382698059 seconds ---
Running Skipgram (dim = 50) with activation rule 'relu'...
0 40086.01563549042
1 39067.42638683319
--- 38.54331088066101 seconds ---
Running Skipgram (dim = 50) with activation rule 'tanh'...
0 40136.364102602005
1 38636.88547039032
--- 38.76637101173401 seconds ---
Running Skipgram (dim = 50) with activation rule 'sigmoid'

# Skipgram additional hidden layer

In [12]:
################################################################################
################### 3. Using Skipgram model (extra layer) ######################
################################################################################

dims = [50, 150, 300]
#dims = [50]
iterations = 2

activation_rules = ["softmax", "elu", "selu", "softplus", "softsign", "relu", "tanh", "sigmoid", "hard_sigmoid", "linear"]
for dimension in dims:
    for activation_rule in activation_rules:
        print("Running Skipgram (dim = %s) with activation rule '%s'..."% (dimension, activation_rule))
        try:
            # set extra_layer = true
            skipgram = getSkipgramModel(dimension, True, activation_rule)
            X, Y = generate_data_skipgram(corpus, window_size, V)

            start_time = time.time()
            for ite in range(iterations):
                loss = 0.
                for x, y in zip(X,Y):
                    loss += skipgram.train_on_batch(x, y)
                print(ite, loss)
            end_time = time.time()

            print("--- %s seconds ---" % (end_time - start_time))
        except:
            print("Error with", activation_rule)

skipgramHiddenLayerWeights = skipgram.get_weights()[0]



Running Skipgram (dim = 50) with activation rule 'softmax'...
0 41252.48097372055
1 39351.12030816078
--- 358.6214940547943 seconds ---
Running Skipgram (dim = 50) with activation rule 'elu'...
0 38090.17261695862
1 37339.52349615097
--- 360.26901602745056 seconds ---
Running Skipgram (dim = 50) with activation rule 'selu'...
0 37969.02503442764
1 37144.54713857174
--- 355.1335310935974 seconds ---
Running Skipgram (dim = 50) with activation rule 'softplus'...
0 36149.578640937805
1 37012.700358986855
--- 354.5474681854248 seconds ---
Running Skipgram (dim = 50) with activation rule 'softsign'...
0 38161.69445538521
1 37472.23338055611
--- 355.6916038990021 seconds ---
Running Skipgram (dim = 50) with activation rule 'relu'...
0 38548.03942465782
1 38201.609553813934
--- 402.5509879589081 seconds ---
Running Skipgram (dim = 50) with activation rule 'tanh'...
0 38081.362594127655
1 37329.15055298805
--- 390.59892201423645 seconds ---
Running Skipgram (dim = 50) with activation rule 'sig

In [18]:
skipgramHiddenLayerWeights.shape
skipgramHiddenLayerWeights[1,:]

array([ 0.08406705,  0.09400582,  0.08651898,  0.07330547,  0.01389818,
        0.05978406,  0.0272817 ,  0.00520233, -0.00388816,  0.04627812,
       -0.03466139, -0.05365833, -0.0472076 ,  0.05588843,  0.08260176,
        0.05304934,  0.04205545,  0.00162497, -0.03770801,  0.01696711,
       -0.04566567,  0.03415297, -0.02130879,  0.073093  , -0.05127802,
        0.01507935,  0.05303619, -0.038434  , -0.02429472, -0.03786976,
       -0.04851353, -0.00866184,  0.04393646,  0.05953187, -0.01863442,
       -0.02132674,  0.02127859,  0.01419815,  0.07605583, -0.02545334,
        0.01741531,  0.05263753,  0.02294727,  0.02463048, -0.00261251,
        0.03773098,  0.02235954,  0.02556502, -0.00441569,  0.01598156,
       -0.05385536,  0.0299219 ,  0.00551344, -0.02750573,  0.05955534,
        0.05302291,  0.04511306,  0.01793033, -0.02903962, -0.07581343,
        0.02622947, -0.02973812,  0.00726978,  0.02505386,  0.01324776,
        0.02039069,  0.08872194,  0.04100237,  0.06498899, -0.03

In [13]:
################################################################################
####### 4a. Implement your own function to perform the analogy task with #######
################################################################################

EMBEDDING_TYPE_1 = "wcoMatrix"
EMBEDDING_TYPE_2 = "distributed"
EMBEDDING_TYPE_4 = "word2vec"


# neighbors is the NearestNeighbor object containing the computed neighbors matrix
# objA is to objB as objC is a objD
# we assume objs are strings
def analogyFunction(neighbors, index, objA, objB, objC, objD, embeddingType = EMBEDDING_TYPE_1, matrix = wcoMatrix):
    
    if embeddingType != EMBEDDING_TYPE_4:
        indexA = index[objA]
        indexB = index[objB]
        indexC = index[objC]
        indexD = index[objD]
    
    if embeddingType == EMBEDDING_TYPE_1 or embeddingType == EMBEDDING_TYPE_2:
        vecA = np.asarray(matrix[indexA,])
        vecB = np.asarray(matrix[indexB,])
        vecC = np.asarray(matrix[indexC,])
        vecD = np.asarray(matrix[indexD,])
    
    if embeddingType == EMBEDDING_TYPE_4 :
        vecA = index.get_vector(objA)
        vecB = index.get_vector(objB)
        vecC = index.get_vector(objC)
        vecD = index.get_vector(objD)

    operationResult = vecA - vecB + vecD
    
    # If the word completely matches, return true
    if np.array_equal(operationResult, vecC) :
        return True
    
    # Otherwise, get the nearest neighbor of the resulting vector
    if embeddingType == EMBEDDING_TYPE_4 :
        finalWord = index.similar_by_vector(operationResult, 1)[0][0]
    else:
        distances, indices = neighbors.kneighbors([operationResult])
        neighborIndex = indices[0, 0]
        finalWord = getKeyFromValue(index, neighborIndex)

    print("\tComparing %s vs. %s (word vs. prediction)" % (objC, finalWord))

    # Return comparison of words
    return ( objC == finalWord )


def makeAnalogyOnFile(neighbors, embeddingType = EMBEDDING_TYPE_1, matrix = wcoMatrix, index = full_index):
    # Read file
    content = []
    with open("analogy_alice.txt") as f:
        content = f.readlines()

    # Remove whitespace characters like '\n' at the end of each line
    content = [x.strip() for x in content] 
    allResults = []
    
    for analogy in content:
        words = analogy.split(" ")
        if len(words) == 4:
            try:
                result = analogyFunction(neighbors, index, words[0], words[1], words[2], words[3], embeddingType, matrix)
                allResults.append(result)
                print("'%s': %s" % (analogy, result))
            except KeyError:
                print("Missing words in the index ")
    return allResults


Comparison performance:

In [14]:
################################################################################
########### 4b. Compare the performance on the analogy task between ############
########### the word embeddings you have trained in 2.1, 2.2 & 2.3  ############
################################################################################

nbrs = NearestNeighbors(n_neighbors = 1).fit(skipgramWeights)
analogyResultsSkipgram1 = makeAnalogyOnFile(nbrs, EMBEDDING_TYPE_2, skipgramWeights, full_index)
print("Score for skipgram embedding: %s" % sum(analogyResultsSkipgram1))


Missing words in the index 
	Comparing usual vs. sudden (word vs. prediction)
'sudden suddenly usual usually': False
	Comparing good vs. better (word vs. prediction)
'bad worse good better': False
	Comparing look vs. looking (word vs. prediction)
'go going look looking': False
	Comparing his vs. her (word vs. prediction)
'he she his her': False
	Comparing his vs. her (word vs. prediction)
'brother sister his her': False
	Comparing look vs. looking (word vs. prediction)
'listen listening look looking': False
	Comparing thinking vs. saying (word vs. prediction)
'saying said thinking thought': False
	Comparing cat vs. cats (word vs. prediction)
'bird birds cat cats': False
	Comparing old vs. good (word vs. prediction)
'good better old older': False
	Comparing quick vs. good (word vs. prediction)
'good better quick quicker': False
	Comparing good vs. large (word vs. prediction)
'large largest good best': False
Missing words in the index 
	Comparing knowing vs. falling (word vs. prediction)

In [196]:
nbrs = NearestNeighbors(n_neighbors = 1).fit(cbowWeights)
analogyResultsSkipgram1 = makeAnalogyOnFile(nbrs, EMBEDDING_TYPE_2, cbowWeights, full_index)
print("Score for cbow embedding: %s" % sum(analogyResultsSkipgram1))


Missing words in the index 
	Comparing usual vs. sudden (word vs. prediction)
'sudden suddenly usual usually': False
	Comparing good vs. better (word vs. prediction)
'bad worse good better': False
	Comparing look vs. looking (word vs. prediction)
'go going look looking': False
	Comparing his vs. with (word vs. prediction)
'he she his her': False
	Comparing his vs. her (word vs. prediction)
'brother sister his her': False
	Comparing look vs. listen (word vs. prediction)
'listen listening look looking': False
	Comparing thinking vs. saying (word vs. prediction)
'saying said thinking thought': False
	Comparing cat vs. cats (word vs. prediction)
'bird birds cat cats': False
	Comparing old vs. good (word vs. prediction)
'good better old older': False
	Comparing quick vs. good (word vs. prediction)
'good better quick quicker': False
	Comparing good vs. large (word vs. prediction)
'large largest good best': False
Missing words in the index 
	Comparing knowing vs. knew (word vs. prediction)
'f

In [15]:
nbrs = NearestNeighbors(n_neighbors = 1).fit(skipgramHiddenLayerWeights)
analogyResultsSkipgram1 = makeAnalogyOnFile(nbrs, EMBEDDING_TYPE_2, skipgramHiddenLayerWeights, full_index)
print("Score for skipgram (extra hideen layer) embedding: %s" % sum(analogyResultsSkipgram1))


Missing words in the index 
	Comparing usual vs. usually (word vs. prediction)
'sudden suddenly usual usually': False
	Comparing good vs. better (word vs. prediction)
'bad worse good better': False
	Comparing look vs. looking (word vs. prediction)
'go going look looking': False
	Comparing his vs. her (word vs. prediction)
'he she his her': False
	Comparing his vs. her (word vs. prediction)
'brother sister his her': False
	Comparing look vs. looking (word vs. prediction)
'listen listening look looking': False
	Comparing thinking vs. saying (word vs. prediction)
'saying said thinking thought': False
	Comparing cat vs. cats (word vs. prediction)
'bird birds cat cats': False
	Comparing old vs. good (word vs. prediction)
'good better old older': False
	Comparing quick vs. good (word vs. prediction)
'good better quick quicker': False
	Comparing good vs. large (word vs. prediction)
'large largest good best': False
Missing words in the index 
	Comparing knowing vs. falling (word vs. prediction

In [198]:
nbrs = NearestNeighbors(n_neighbors = 1).fit(cbowHiddenLayerWeights)
analogyResultsSkipgram1 = makeAnalogyOnFile(nbrs, EMBEDDING_TYPE_2, cbowHiddenLayerWeights, full_index)
print("Score for cbow (extra hideen layer) embedding: %s" % sum(analogyResultsSkipgram1))


NameError: name 'cbowHiddenLayerWeights' is not defined

In [ ]:
################################################################################
############ 4c. Visualize your results and interpret your results #############
################################################################################



In [199]:
################################################################################
############# 5. Use the word co-occurence matrix from Question 1 ##############
############# Compare the performance on the analogy task.        ##############
################################################################################
    
nbrs = NearestNeighbors(n_neighbors = 1).fit(wcoMatrix)
analogyResultsWcoMatrix = makeAnalogyOnFile(nbrs, EMBEDDING_TYPE_1, wcoMatrix, full_index)
print("Score for w-w co-ocurrence matrix embedding: %s" % sum(analogyResultsWcoMatrix))

Missing words in the index 
	Comparing usual vs. usually (word vs. prediction)
'sudden suddenly usual usually': False
	Comparing good vs. bad (word vs. prediction)
'bad worse good better': False
	Comparing look vs. looking (word vs. prediction)
'go going look looking': False
	Comparing his vs. particular (word vs. prediction)
'he she his her': False
	Comparing his vs. her (word vs. prediction)
'brother sister his her': False
	Comparing look vs. looking (word vs. prediction)
'listen listening look looking': False
	Comparing thinking vs. continued (word vs. prediction)
'saying said thinking thought': False
	Comparing cat vs. cats (word vs. prediction)
'bird birds cat cats': False
	Comparing old vs. good (word vs. prediction)
'good better old older': False
	Comparing quick vs. good (word vs. prediction)
'good better quick quicker': False
	Comparing good vs. large (word vs. prediction)
'large largest good best': False
Missing words in the index 
	Comparing knowing vs. knew (word vs. predic

In [ ]:
################################################################################
############## 7b. Performance on the analogy task Glove-word2vec ##############
################################################################################

# Get labels 
sorted_labels = sorted(full_index.items(), key=operator.itemgetter(1))
sorted_labels.insert(0, ('', 0))
labels_only = [i[0] for i in sorted_labels]
labels = np.asarray(labels_only)

##################################### CBOW #####################################

# Transform data to 2d
tsne = TSNE(n_components=2, verbose=1)
transformed_weights = tsne.fit_transform(cbowWeights)

# Visualize
plot_embedding(transformed_weights, labels, "Embedding for simple CBOW")


################################### Skipgram ###################################

# Transform data to 2d
tsne = TSNE(n_components=2, verbose=1)
transformed_weights = tsne.fit_transform(skipgramWeights)

# Visualize
plot_embedding(transformed_weights, labels, "Embedding for simple Skipgram")


############################## CBOW (extra layer) ##############################

# Transform data to 2d
tsne = TSNE(n_components=2, verbose=1)
transformed_weights = tsne.fit_transform(cbowHiddenLayerWeights)

# Visualize
plot_embedding(transformed_weights, labels, "Embedding for CBOW (extra layer)")


############################ Skipgram (extra layer) ############################

# Transform data to 2d
tsne = TSNE(n_components=2, verbose=1)
transformed_weights = tsne.fit_transform(skipgramHiddenLayerWeights)

# Visualize
plot_embedding(transformed_weights, labels, "Embedding for Skipgram (extra layer)")



In [ ]:
################################################################################
############# 5. Use the word co-occurence matrix from Question 1. #############
################################################################################

# Compare the performance on the analogy task with the performance of your trained word embeddings.  
analogyResultsWcoMatrix = makeAnalogyOnFile()

print("\nScore for w-w co-ocurrence matrix embedding: %s" % sum(analogyResultsWcoMatrix))


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

**6. Discuss**

Jorge > We can pick ideas from the paper and the book. Should be easy

#### What are the main advantages of CBOW and Skipgram?


#### What is the advantage of negative sampling?


#### What are the main drawbacks of CBOW and Skipgram?




#### 7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300

In [ ]:
################################################################################
############## 7a. Load pre-trained embeddings on large corpuses  ##############
################################################################################

#load pretrained word embeddings of word2vec
path_word2vec = "./GoogleNews-vectors-negative300.bin"
word2vec = KeyedVectors.load_word2vec_format(path_word2vec, binary=True)


In [71]:
#load pretraind word embeddings of Glove

#path = "./glove.6B/glove.6B.300d_converted.txt"
path = "./glove.6B/glove.6B.300d.txt"
path2 = "./glove_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path2, binary=False)


In [200]:
################################################################################
############# 7b. Performance on the analogy task Google-word2vec  #############
################################################################################

analogyResultsWord2Vec = makeAnalogyOnFile(neighbors = None, embeddingType = EMBEDDING_TYPE_4, matrix = None, index = word2vec)
print("Score for GoogleNews embedding: %s" % sum(analogyResultsWord2Vec))


	Comparing pleasant vs. unpleasant (word vs. prediction)
'happy unhappy pleasant unpleasant': False
	Comparing usual vs. usually (word vs. prediction)
'sudden suddenly usual usually': False
	Comparing good vs. good (word vs. prediction)
'bad worse good better': True
	Comparing look vs. looking (word vs. prediction)
'go going look looking': False
	Comparing his vs. his (word vs. prediction)
'he she his her': True
	Comparing his vs. his (word vs. prediction)
'brother sister his her': True
	Comparing look vs. looking (word vs. prediction)
'listen listening look looking': False
	Comparing thinking vs. thought (word vs. prediction)
'saying said thinking thought': False
	Comparing cat vs. cat (word vs. prediction)
'bird birds cat cats': True
	Comparing old vs. older (word vs. prediction)
'good better old older': False
	Comparing quick vs. quicker (word vs. prediction)
'good better quick quicker': False
	Comparing good vs. good (word vs. prediction)
'large largest good best': True
	Comparing 

In [201]:
################################################################################
############## 7b. Performance on the analogy task Glove-word2vec ##############
################################################################################

analogyResultsGlove = makeAnalogyOnFile(neighbors = None, embeddingType = EMBEDDING_TYPE_4, matrix = None, index = glove)
print("Score for Glove embedding: %s" % sum(analogyResultsWord2Vec))


	Comparing pleasant vs. unpleasant (word vs. prediction)
'happy unhappy pleasant unpleasant': False
	Comparing usual vs. usually (word vs. prediction)
'sudden suddenly usual usually': False
	Comparing good vs. good (word vs. prediction)
'bad worse good better': True
	Comparing look vs. looking (word vs. prediction)
'go going look looking': False
	Comparing his vs. his (word vs. prediction)
'he she his her': True
	Comparing his vs. his (word vs. prediction)
'brother sister his her': True
	Comparing look vs. looking (word vs. prediction)
'listen listening look looking': False
	Comparing thinking vs. thought (word vs. prediction)
'saying said thinking thought': False
	Comparing cat vs. cats (word vs. prediction)
'bird birds cat cats': False
	Comparing old vs. older (word vs. prediction)
'good better old older': False
	Comparing quick vs. quicker (word vs. prediction)
'good better quick quicker': False
	Comparing good vs. best (word vs. prediction)
'large largest good best': False
	Compari

In [ ]:
################################################################################
###################### 7. Visualize embeddings (word2vec) ######################
################################################################################

################################## GoogleNews ##################################

#Transform data to 2d
tsne = TSNE(n_components=2, verbose=1)
transformed_weights = tsne.fit_transform(word2vec.vectors)

# Get labels 
sorted_labels = sorted(full_index.items(), key=operator.itemgetter(1))
sorted_labels.insert(0, ('', 0))
labels_only = [i[0] for i in sorted_labels]
labels = np.asarray(labels_only)

# Visualize
plot_embedding(transformed_weights, labels, "Embedding for word2vec (Google)")

#################################### Glove ####################################

#Transform data to 2d
tsne = TSNE(n_components=2, verbose=1)
transformed_weights = tsne.fit_transform(glove.vectors)

# Get labels 
sorted_labels = sorted(full_index.items(), key=operator.itemgetter(1))
sorted_labels.insert(0, ('', 0))
labels_only = [i[0] for i in sorted_labels]
labels = np.asarray(labels_only)

# Visualize
plot_embedding(transformed_weights, labels, "Embedding for word2vec (Google)")



[t-SNE] Computing 91 nearest neighbors...


Comparison performance with your own trained word embeddings